In [1]:
import torch 
import open3d as o3d
import torch.nn as nn 
import os
import trimesh
import numpy as np
import math
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_point_cloud_by_file_extension(file_name  = "/home/ai/Desktop/Preimage_Implicit_DLTaskData/armadillo_10000.xyz"):

    ext = file_name.split('.')[-1]

    if ext == "npz" or ext == "npy":
        point_set = torch.tensor(np.load(file_name)).float()
    else:
        point_set = torch.tensor(trimesh.load(file_name, "xyz").vertices).float()

    return point_set

In [3]:
data = load_point_cloud_by_file_extension()

In [4]:
x2,y2,z2= torch.max(data, axis = 0).values.numpy()
x2,y2,z2

(1.26818, 1.9414015, 0.78549665)

In [5]:
x1, y1, z1 = torch.min(data, axis=0).values.numpy()
x1,y1,z1

(-1.2706181, -1.056159, -0.98085314)

In [7]:
def get_sigma_smaples(bp_size):
    
    n = bp_size

    xs = 1.5*((x2 - x1)*np.random.rand(n) + x1)
    ys = 1.5*((y2 - y1)*np.random.rand(n) + y1)
    zs = 1.5*((z2 - z1)*np.random.rand(n) + z1)

    return torch.tensor(list(zip(xs,ys,zs)))

In [118]:
np.max(xs), np.min(xs)

(1.9014745784127072, -1.903348471737519)

In [121]:
##Ball 
# x=rsinϕcosθ
# y=rsinϕsinθ
# z=rcosϕ

In [42]:
def get_sigma_smaples(bp_size):
    
    n = bp_size

    xs = 1.5*((x2 - x1)*np.random.rand(n) + x1)
    ys = 1.5*((y2 - y1)*np.random.rand(n) + y1)
    zs = 1.5*((z2 - z1)*np.random.rand(n) + z1)

    return torch.tensor(list(zip(xs,ys,zs)))
                             
                             
def get_ball_points(data):
                             
    r = 0.01
    
    all_ball_points = []

    for xo,yo,zo in data:
        
        x_sb, y_sb, z_sb = [],[],[]
        temp = []
        for i in range(0,50):

            theta = random.uniform(*(0, 360))
            phi   = random.uniform(*(0, 360))

            x_sb.append(xo + math.sin(phi)* math.cos(theta)*r)
            y_sb.append(yo + math.sin(phi)* math.sin(theta)*r)
            z_sb.append(zo + math.cos(phi)*r)
        
        temp = np.column_stack((np.array(x_sb), np.array(y_sb), np.array(z_sb)))
        all_ball_points.append(temp)
    
    return   all_ball_points

In [27]:
all_ball_points = get_ball_points(data)

In [28]:
np.array(all_ball_points).shape

(10098, 50, 3)

In [44]:
indices = torch.tensor(np.random.choice(data.shape[0], 2048 , False))

In [47]:
ball_pts = torch.tensor(all_ball_points)[indices].reshape((2048*50,3))

In [48]:
sigma_pts = get_sigma_smaples(ball_pts.shape[0])

In [49]:
sigma_pts.shape

torch.Size([102400, 3])